In [1]:
! pip install numpy

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [3]:
# Function to extract Product Title
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})
        
        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

# Function to extract Product Price
def get_price(soup):

    try:
        price = soup.find("span", attrs={"class":'a-price aok-align-center'}).find("span",attrs={"class":'a-offscreen'}).string.strip()

    except AttributeError:

        # try:
        #     # If there is some deal price
        #     price = soup.find("span", attrs={'id':'priceblock_dealprice'}).string.strip()

        # except:
        price = ""

    return price

# Function to extract Product Rating
def get_rating(soup):

    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5 cm-cr-review-stars-spacing-big'}).string.strip()
    
    except AttributeError:
        try:
            rating = soup.find("span", attrs={"class":'a-icon-alt'}).string.strip()
        except:
            rating = ""	

    return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""	

    return review_count

# Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"	

    return available


In [14]:
if __name__ == '__main__':

    # user agent 
    HEADERS=({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36','Accept-Language':'en-US, en;q=0.5'})

    # The webpage URL
    URL = "https://www.amazon.com/s?k=playstation+5&crid=2RF1FEJBTOHAC&sprefix=pla%2Caps%2C381&ref=nb_sb_ss_ts-doa-p_1_3"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)
    

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))
    

    d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}
    
    # Loop for extracting product details from each link 
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.com" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))

    
    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)


<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


ConnectionError: HTTPSConnectionPool(host='www.amazon.comhttps', port=443): Max retries exceeded with url: //aax-us-iad.amazon.com/x/c/RMY_ymylH7uYdeULr8gOKXoAAAGMM27ZQwEAAAH2AQBvbm9fdHhuX2JpZDQgICBvbm9fdHhuX2ltcDEgICCPBDA5/https://www.amazon.com/Cooling-Controller-Charging-Playstation-Accessories/dp/B09QGK2LW3/ref=sxin_36_sbv_search_btf?content-id=amzn1.sym.5795aee5-71f0-4369-b632-e8c78407f2cf%3Aamzn1.sym.5795aee5-71f0-4369-b632-e8c78407f2cf&crid=2RF1FEJBTOHAC&cv_ct_cx=playstation+5&keywords=playstation+5&pd_rd_i=B09QGK2LW3&pd_rd_r=39b2b1af-0b0f-466b-b10a-6212ced109d1&pd_rd_w=Jl6yk&pd_rd_wg=nYuSf&pf_rd_p=5795aee5-71f0-4369-b632-e8c78407f2cf&pf_rd_r=PM4MB5P5RR584RTY9NNC&qid=1701669951&sbo=RZvfv%2F%2FHxDF%2BO5021pAnSA%3D%3D&sprefix=pla%2Caps%2C381&sr=1-1-5190daf0-67e3-427c-bea6-c72c1df98776 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000025D885DFA90>: Failed to resolve 'www.amazon.comhttps' ([Errno 11001] getaddrinfo failed)"))